# 日本語wordnet
参考サイト<br>
https://qiita.com/pocket_kyoto/items/f2382b9821d32624e6bc<br>
https://github.com/EmulsionBondo/jwordnet/blob/master/SearchUpperConceptWords.py

In [1]:
import sqlite3
import sys
conn = sqlite3.connect("wnjpn.db")

In [2]:
# 上位-下位の関係にある概念の抽出
hierarchy_dict = {}  # key:上位語(String), value:下位語(List of String)
n_term_set = set()  # 下位語に含まれる単語集合

In [3]:
class node:
    def __init__(self, name, children=None):
        self.name = name  # String
        self.children = children  # List of Class node

    # 結果表示用
    def display(self, indent = 0):
        if self.children != None:
            print(' '*indent + self.name)
            for c in self.children:
                c.display(indent+1)
        else:
            print(' '*indent + self.name)

In [4]:
def search_upper_concept_words(word, hierarchy_dict):
    """
    特定の単語を入力とした時に、上位語を検索する関数
    :param word: 検索する単語
    :param hierarchy_dict: 上位語
    """
    cur = conn.execute(f"select wordid from word where lemma='{word}'")
    word_id = 99999999  #temp
    for row in cur:
        word_id = row[0]
    
    if word_id == 99999999: # Wordnetに存在する語であるかの判定
        print(f'「{word}」は存在しない単語です。')
    else:
        print(f'「{word}」の上位概念を出力します。')
    
    # 入力された単語を含む概念を検索
    cur = conn.execute(f"select synset from sense where wordid='{word_id}'")
    synsets = []
    for row in cur:
        synsets.append(row[0])
    
    for synset in synsets:
        # 上位語を取得する対象のsynsetを表示
        print(f'------------------{synset_name_dict[synset]}---------------------')
        #現在のsynsetの上位語をすべて取得するため，tmp_synsetに代入
        tmp_synset=synset
        # synsetの上位語を最上位まで列挙
        while(tmp_synset in hierarchy_dict.values()):
            # エラーが発生=最上位を取得したら終了
            try:
                print(synset_name_dict[hierarchy_dict[tmp_synset]])
                tmp_synset = hierarchy_dict[tmp_synset]
            except:
                break

In [5]:
# 下位-上位の関係にある概念の抽出
cur = conn.execute("select synset1,synset2 from synlink where link='hypo'")

In [6]:
hierarchy_dict = {}  # key:下位語(String), value:上位語(String)

In [7]:
for row in cur:
    b_term = row[0] # 下位概念のID
    n_term = row[1] # 上位概念のID
    
    if n_term not in hierarchy_dict:
        hierarchy_dict[n_term] = b_term

In [8]:
# synset(概念)のIDから、概念の名称に変換する辞書の作成
synset_name_dict = {}  # key:synsetのID, value:synsetの名称
cur = conn.execute("select synset,name from synset")

In [9]:
for row in cur:
    synset_name_dict[row[0]] = row[1]

In [10]:
search_upper_concept_words('猫', hierarchy_dict)

「猫」の上位概念を出力します。
------------------true_cat---------------------
feline
carnivore
placental
mammalian
vertebrate
chordate
animate_being
being
living_thing
whole
physical_object
physical_entity
entity


In [16]:
hierarchy_dict.__sizeof__()

5242952

In [13]:
test = '顛落'

In [14]:
search_upper_concept_words(test, hierarchy_dict)

「顛落」の上位概念を出力します。
------------------fall---------------------
